In [1]:
from keras import utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import (
    Activation,
    Dense,
    Dropout,
    Flatten, 
    Conv2D,
    MaxPooling2D)
import numpy as np
import pandas as pd
import tensorflow as tf

Using TensorFlow backend.


In [40]:
# Data structure :'Word POS_tag syntactic_chunk_tag named_entity_tag \n'
# Order a word = prefixe 2, 3, 4, 5 then suffixe 2, 3, 4, 5, capitals, new_sentence, numbers, middle_dashes, appostrophies

# train = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.train', 'at')
# testa = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.testa')
# testb = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.testb')
  
def get_words(path, pos): # Add a 'r' before the path for raw string, pos between 0 and 3 see data structure
    data = open(path)
    list_words = []
    new_data = data.readlines()[2:] # Skip first 2 lines
    mylist = list(new_data) # Make the data readable
    for i in range(500): # TODO : CHANGES 20 TO len(mylist)
        if splitter(mylist[i], ' ', pos): # it return None if the line est empty
            list_words.append((splitter(mylist[i], ' ', pos)))
    return list_words

def splitter(line, separator, pos): # pos between 0 and 3 for CONLL2003
    if line.isspace(): # Removing blank lines
        return
    new_sentence = []
    for word in line.split(separator):
        new_sentence.append(word)
    return new_sentence[pos]

def get_dictionary(list_of_words): # Return a dict with how many times a word appears
    dicti = {}
    for word in list_of_words:
        if word in dicti:
            dicti[word] = dicti[word] + 1
        else:
            dicti[word] = 1
    return dicti

def get_reduced_dictionary(dicti, limit): # return a dict with key >= limit
    for v, k in list(dicti.items()):
        if k <= limit: # We don't want to take into account words that doesn't appear enough (limit)
            del dicti[v]
    return dicti

def get_prefixe(reduced_dict, prefixe_length):
    #TODO : remove all ponctuations signs
    list_prefixes = []
    for word in reduced_dict:
        prefixe = word[:prefixe_length]
        list_prefixes.append(prefixe)
#     print('list prefixes', list_prefixes)
    return list_prefixes

def get_suffixe(reduced_dict, suffixe_length):
    #TODO : remove all ponctuations signs
    list_suffixes = []
    for word in reduced_dict:
        util_position = suffixe_length * (-1)
        suffixe = word[util_position:]
        list_suffixes.append(suffixe)
#     print('list suffixes', list_suffixes)
    return list_suffixes

def has_capital(string):
    if any(char.isupper() for char in string):
        return 1
    return 0


def get_capital(list_words):
    # This function returns 2 lists:
    list_capitals = []   # 1/ If the word have a capital
    list_first_word = [] # 2/ If the word is the 1st word of the sentence
    chara_end_sentence = ['.', '!', '?', '...'] # TODO : Check for '...' if it should be in this list
    flag_new_sentence = True
    for word in list_words:
        letter = word[0] # First 'letter' of the word; can be number or special chara
        if letter.isupper():
            list_capitals.append(1)
            if flag_new_sentence:
                list_first_word.append(1)
            else:
                list_first_word.append(0)
        else:
            list_capitals.append(0)
            list_first_word.append(0)
        if letter in chara_end_sentence:
            flag_new_sentence = True
            continue
        flag_new_sentence = False
#     print(list_capitals, list_first_word)
    return (list_capitals, list_first_word)


def has_numbers(string):
    if any(char.isdigit() for char in string):
        return 1
    return 0


def get_number(list_words): # Return array if there is a number in the word
    list_numbers = []
    for word in list_words:
        if has_numbers(word):
            list_numbers.append(1)
        else:
            list_numbers.append(0)
#     print(list_numbers)
    return list_numbers

def has_middle_dash(string):
    if any(char == '-' for char in string):
        return 1
    return 0

def get_middle_dash(list_words):
    list_middle_dashes = []
    for word in list_words:
        if has_middle_dash(word):
            list_middle_dashes.append(1)
        else:
            list_middle_dashes.append(0)
#     print(list_middle_dashes)
    return list_middle_dashes


def has_appostrophy(string):
    if any(char == '\'' for char in string):
        return 1
    return 0
    
def get_appostrophy(list_words):
    list_appostrophy = []
    for word in list_words:
        if has_appostrophy(word):
            list_appostrophy.append(1)
        else:
            list_appostrophy.append(0)
#     print(list_appostrophy)
    return list_appostrophy



def get_features():
    list_of_words = get_words(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.train', 0)
#     print(list_of_words)
    dicti = get_dictionary(list_of_words)
    reduced_dict = get_reduced_dictionary(dicti, 5)
    list_prefixe_2 = get_prefixe(reduced_dict, 2)
    list_prefixe_3 = get_prefixe(reduced_dict, 3)
    list_prefixe_4 = get_prefixe(reduced_dict, 4)
    list_prefixe_5 = get_prefixe(reduced_dict, 5)
    list_suffixe_2 = get_suffixe(reduced_dict, 2)
    list_suffixe_3 = get_suffixe(reduced_dict, 3)
    list_suffixe_4 = get_suffixe(reduced_dict, 4)
    list_suffixe_5 = get_suffixe(reduced_dict, 5)
    list_capitals, list_new_sentence = get_capital(list_of_words)
    list_numbers = get_number(list_of_words)
    list_middle_dashes = get_middle_dash(list_of_words)
    list_appostrophies = get_appostrophy(list_of_words)
    
    get_array_for_sentence('Je suis un homme de 23 ans .', list_prefixe_2, list_prefixe_3, list_prefixe_4, list_prefixe_5,
                           list_suffixe_2, list_suffixe_3, list_suffixe_4, list_suffixe_5)

#TODO REMOVE ALL DOCSTART LINES    
    


# Now we need to create a vector to describe the words

def get_vector_for_prefixe(word, prefixe_length, list_prefixe):
    vector_prefixe = [] # It should be of size get_prefixe()
    prefixe = word[:prefixe_length]
    for pref in list_prefixe:
        if pref == prefixe:
            vector_prefixe.append(1)
        else:
            vector_prefixe.append(0)
#     print('vector prefixe', vector_prefixe)
    return vector_prefixe


def get_vector_for_suffixe(word, suffixe_length, list_suffixe):
    vector_suffixe = []
    util_position = suffixe_length * (-1)
    suffixe = word[util_position:]
    for suf in list_suffixe:
        if suf == suffixe:
            vector_suffixe.append(1)
        else:
            vector_suffixe.append(0)
#     print('vector suffixe', vector_suffixe)
    return vector_suffixe

def get_array_for_sentence(sentence, list_prefixe_2, list_prefixe_3, list_prefixe_4, list_prefixe_5,
                          list_suffixe_2, list_suffixe_3, list_suffixe_4, list_suffixe_5, 
#                           list_capitals, list_new_sentence, list_numbers, list_middle_dashes, list_appostrophies
                          ):
    output = np.array([[]])
    word_description = []
    for word in sentence:
#         word_description.append(get_vector_for_prefixe(word, 2, list_prefixe_2))
#         word_description.append(get_vector_for_prefixe(word, 3, list_prefixe_3))
#         word_description.append(get_vector_for_prefixe(word, 4, list_prefixe_4))
#         word_description.append(get_vector_for_prefixe(word, 5, list_prefixe_5))
#         word_description.append(get_vector_for_suffixe(word, 2, list_suffixe_2))
#         word_description.append(get_vector_for_suffixe(word, 3, list_suffixe_3))
#         word_description.append(get_vector_for_suffixe(word, 4, list_suffixe_4))
#         word_description.append(get_vector_for_suffixe(word, 5, list_suffixe_5))
#         word_description.append(has_capital(word))
#         word_description.append(has_numbers(word))
#         word_description.append(has_middle_dash(word))
#         word_description.append(has_appostrophy(word))

        word_description = get_vector_for_prefixe(word, 2, list_prefixe_2)
        word_description = word_description + get_vector_for_prefixe(word, 3, list_prefixe_3)
        word_description = word_description + get_vector_for_prefixe(word, 4, list_prefixe_4)
        word_description = word_description + get_vector_for_prefixe(word, 5, list_prefixe_5)
        word_description = word_description + get_vector_for_suffixe(word, 2, list_suffixe_2)
        word_description = word_description + get_vector_for_suffixe(word, 3, list_suffixe_3)
        word_description = word_description + get_vector_for_suffixe(word, 4, list_suffixe_4)
        word_description = word_description + get_vector_for_suffixe(word, 5, list_suffixe_5)
        word_description = word_description + [has_capital(word)]
        word_description = word_description + [has_numbers(word)]
        word_description = word_description + [has_middle_dash(word)]
        word_description = word_description + [has_appostrophy(word)]
        print('word', word_description)
        output = np.append(output, np.array([word_description])) #this create a big array
    print(output, output.shape)
    return output
        


get_features()


#     TODO : CONCATENATE THE LIST VECTOR TO HAVE THE FINAL DESCRIPTION IN A WORD IN 1 LIST 

word [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
word [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
word [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
word [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
test={
    "date": 1999,
    "bojour": 3, 
    "popo": 15
}
for x in test:
    print(x)
print(test)


date
bojour
popo
{'date': 1999, 'bojour': 3, 'popo': 15}


In [13]:
line = np.array([[0, 0], [0, 0]])
np.append(line, [3, 2], axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

[[0 1 2 3]]
